In [56]:
import pandas as pd
from pymongo import MongoClient
import json

In [3]:
client = MongoClient('mongodb://localhost:27017')
db = client.mfers_collection
mfer = db.mfer

Values we need to get are as follows:
- event_timestamp
- transaction
    - user
        - username
    - address


In [134]:

i = 0
a = 0
for mf in mfer.find():
# Gets Timeline data for transfers for a single token_id
    for data in mf['asset_events']:
        i += 1
        if i == 1:
            df = pd.json_normalize(data)
        else:
            temp = pd.json_normalize(data)
            df = pd.concat([df,temp])

# Adds the dataframes together with the other data
        print(df)

  asset_bundle event_type      event_timestamp auction_type total_price  \
0         None   transfer  2022-04-23T06:12:41         None        None   

  payment_token                created_date quantity approved_account  \
0          None  2022-04-23T06:13:10.346609        1             None   

  bid_amount  ...                       transaction.transaction_hash  \
0       None  ...  0x13c91e4634645b750b0a3e08adb76c2c8eef1d20c23e...   

  transaction.transaction_index from_account.user.username  \
0                            58                     stimpy   

                        from_account.profile_img_url  \
0  https://storage.googleapis.com/opensea-static/...   

                         from_account.address from_account.config  \
0  0xd9df7441f87c197887cf438eebe3368637e76a78                       

  to_account.user.username                         to_account.profile_img_url  \
0           stimpys_cellar  https://storage.googleapis.com/opensea-static/...   

                 